In [42]:
pip install sklearn_crfsuite

In [43]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [44]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
#reading dataset and taking 50k values for input
df = pd.read_csv('ner_dataset.csv', encoding = "ISO-8859-1")
df = df[:50000]
df

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
49995,NaN,pushed,VBN,O
49996,NaN,by,IN,O
49997,NaN,South,NNP,B-geo
49998,NaN,Africa,NNP,I-geo


In [46]:
#Checking Null
df.isnull().sum()

Sentence #    47730
Word              0
POS               0
Tag               0
dtype: int64

In [47]:
#filling sentence values with the sentence value of first word.
df = df.fillna(method='ffill')

In [48]:
df

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
49995,Sentence: 2270,pushed,VBN,O
49996,Sentence: 2270,by,IN,O
49997,Sentence: 2270,South,NNP,B-geo
49998,Sentence: 2270,Africa,NNP,I-geo


In [49]:
(df['Word'][:23])

0         Thousands
1                of
2     demonstrators
3              have
4           marched
5           through
6            London
7                to
8           protest
9               the
10              war
11               in
12             Iraq
13              and
14           demand
15              the
16       withdrawal
17               of
18          British
19           troops
20             from
21             that
22          country
Name: Word, dtype: object

In [50]:
#IOB tags and their counts
labels=df.groupby('Tag').size().reset_index(name='counts')
labels

,Tag,counts
0,B-art,48
1,B-eve,39
2,B-geo,1490
3,B-gpe,968
4,B-nat,18
5,B-org,959
6,B-per,789
7,B-tim,880
8,I-art,27
9,I-eve,33


In [51]:
#Making Dataset into array of sentences with their IOB and POS tags
class SentenceGetter(object): 
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences
sentences[0]

[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

In [52]:
#generating input features for the model.
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [53]:
#splitting test and train data
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [54]:
X[0][2]

{'+1:postag': 'VBP',
 '+1:postag[:2]': 'VB',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'have',
 '-1:postag': 'IN',
 '-1:postag[:2]': 'IN',
 '-1:word.istitle()': False,
 '-1:word.isupper()': False,
 '-1:word.lower()': 'of',
 'bias': 1.0,
 'postag': 'NNS',
 'postag[:2]': 'NN',
 'word.isdigit()': False,
 'word.istitle()': False,
 'word.isupper()': False,
 'word.lower()': 'demonstrators',
 'word[-2:]': 'rs',
 'word[-3:]': 'ors'}

In [55]:
X[0]

[{'+1:postag': 'IN',
  '+1:postag[:2]': 'IN',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.lower()': 'of',
  'BOS': True,
  'bias': 1.0,
  'postag': 'NNS',
  'postag[:2]': 'NN',
  'word.isdigit()': False,
  'word.istitle()': True,
  'word.isupper()': False,
  'word.lower()': 'thousands',
  'word[-2:]': 'ds',
  'word[-3:]': 'nds'},
 {'+1:postag': 'NNS',
  '+1:postag[:2]': 'NN',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.lower()': 'demonstrators',
  '-1:postag': 'NNS',
  '-1:postag[:2]': 'NN',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:word.lower()': 'thousands',
  'bias': 1.0,
  'postag': 'IN',
  'postag[:2]': 'IN',
  'word.isdigit()': False,
  'word.istitle()': False,
  'word.isupper()': False,
  'word.lower()': 'of',
  'word[-2:]': 'of',
  'word[-3:]': 'of'},
 {'+1:postag': 'VBP',
  '+1:postag[:2]': 'VB',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.lower()': 'have',
  '-1:postag': 'I

In [56]:
#training the model
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.06146442424219609,
    c2=0.30343585910230675,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.06146442424219609, c2=0.30343585910230675,
    calibration_candidates=None, calibration_eta=None,
    calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [57]:
#predictions in test data
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,average='weighted', labels=labels['Tag'])

0.9561293122804848

In [58]:
y_pred[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [59]:
#metrics and transition probabilities
#print(metrics.flat_classification_report(y_test, y_pred))
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Most likely transitions")
print_transitions(Counter(crf.transition_features_).most_common(5))
print("\nMost unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-5:])

Most likely transitions
B-geo  -> I-geo   4.883748
I-org  -> I-org   4.691151
B-org  -> I-org   4.589987
B-per  -> I-per   4.589313
I-tim  -> I-tim   4.456317

Most unlikely transitions:
B-per  -> B-per   -1.628133
O      -> I-per   -2.064868
O      -> I-geo   -2.567451
O      -> I-tim   -2.604587
O      -> I-org   -2.636209


In [66]:
#Function to print answer based on results
ent={'geo':'Geographical Location','per':'Person','tim':'Time','gpe':'Geo Political Entity','org':'Organisation','nat':'Natural Phenomenon','art':'Artifact','eve':'Event'}
def sentence_result(sentence):
  result=senttoinput(sentence)
  sent=nltk.word_tokenize(sentence)
  for i in range(0,1):
    for j in range(0,len(result[i])):
      tg=result[i][j]
      if ((j+1)<len(result[i])):
        tgnxt=result[i][j+1]
      else:
        tgnxt=" "
      if tg[0]=='B' and tgnxt[0]=='I':
        print(sent[j]+" "+sent[j+1],end=" - ")
        j=j+1
        print(ent[str(tg[2:])])
      elif tg[0]=='B':
        print(sent[j],end=" - ")
        print(ent[str(tg[2:])])

In [67]:
#Function to convert input to features for prediction
def senttoinput(sent):
  tag=tok(sent)
  tokens=[]
  pos=[]
  sent=[]
  iob=[]
  dftemp=[]
  for i in tag:
    tokens.append(i[0])
    pos.append(i[1])
    sent.append("Sentence: 1")
    iob.append("O")
  dftemp=[sent,tokens,pos,iob]
  df = pd.DataFrame(list(zip(sent,tokens,pos,iob)),columns =['Sentence #', 'Word','POS','Tag'])
  getter = SentenceGetter(df)
  sentences = getter.sentences  
  test=[sent2features(s) for s in sentences]
  result=crf.predict(test)
  return result
  
#POS tagging function
def tok(sent):
    wordsList = nltk.word_tokenize(sent)
    tagged = nltk.pos_tag(wordsList)
    return(tagged)

In [70]:
def print_result(par):
  sen=par.split(".")
  for i in sen:
    sentence_result(i)

In [71]:
print_result("John William went to United States on Tuesday. Mukunth went to Saudi Arabia on January")

John William - Person
United States - Geographical Location
Tuesday - Time
Mukunth - Person
Saudi Arabia - Geographical Location
January - Time
